In [1]:
from scipy import signal as sig

# imports from custom library
import sys
sys.path.append('../../')
import matplotlib.pyplot as plt
plt.rc('text', usetex=True)
from mlrefined_libraries import convnets_library as convlib
from mlrefined_libraries import basics_library as baslib
from mlrefined_libraries import superlearn_library as superlearn
from mlrefined_libraries import multilayer_perceptron_library as network_lib

import autograd.numpy as np
from autograd import grad as compute_grad   

import autograd.numpy as np
import numpy as npo

import pandas as pd
import cv2
import csv
import pickle
import glob
import time
import copy
from datetime import datetime 

#this is needed to compensate for matplotlib notebook's tendancy to blow up images when plotted inline
%matplotlib notebook
from matplotlib import rcParams
rcParams['figure.autolayout'] = True

%load_ext autoreload
%autoreload 2

# Test transformation on face images

Load in data, transform via original method, transform via new method, compare features to make sure everything looks good.

## Old way

In [2]:
 def load_data(datapath):
    # load in data
    data = np.loadtxt(datapath,delimiter = ',')

    # import data and reshape appropriately
    X = data[:,:-1]
    y = data[:,-1]
    y.shape = (len(y),1)
    
    X_square = np.zeros((len(y),28,28))
    for i in range(0,len(y)):
        X_square[i,:,:] = np.reshape(X[i,:],(28,28),1)
    
    # pad data with ones for more compact gradient computation
    o = np.ones((np.shape(X)[0],1))
    X = np.concatenate((o,X),axis = 1)
    X = X.T
    
    return X,X_square,y

In [8]:
# load data
datapath = '../../mlrefined_datasets/convnet_datasets/feat_face_data.csv'
X,image_tensor, y = load_data(datapath)

In [9]:
image_tensor = image_tensor[:100]

In [10]:
# load kernels
kernels = convlib.image_viz.load_kernels()

# params
sliding_window_size = (6,6) 
stride=3
pooling_func= 'max'

# get number of images in the dataset
num_images = np.shape(image_tensor)[0]
        
# a test run to find the number of features with the params above
test = convlib.image_viz.make_feat(image_tensor[0,:,:], kernels, sliding_window_size=sliding_window_size, stride=stride)
num_features = np.shape(test)[0]

In [11]:
# start timer
startTime= datetime.now() 

# run old method
feat = []
for i in range(0,num_images):
    # extract features
    f = convlib.image_viz.make_feat(image_tensor[i,:,:], kernels, sliding_window_size=sliding_window_size,
                                            stride=stride, pooling_func=pooling_func) 
    # store it
    feat.append(f)
    
# convert to array
feat = np.asarray(feat)

# time for measurment
timeElapsed=datetime.now()-startTime 

print('Time elpased (hh:mm:ss.ms) {}'.format(timeElapsed))

Time elpased (hh:mm:ss.ms) 0:00:00.326260


In [12]:
np.shape(image_tensor)

(100, 28, 28)

In [13]:
np.shape(feat)

(100, 512)

# Most naive image version

In [36]:
### parameters for naive convolution ###
conv_function = lambda window: np.sum(kernel*window)

# pooling / downsampling parameters
pool_function = lambda window: np.max(window) 

# activation 
activation = lambda window: np.maximum(0,window)

# pad image with appropriate number of zeros for convolution
def pad_image(image,kernel_size):
    odd_nums = np.array([int(2*n + 1) for n in range(100)])
    pad_val = np.argwhere(odd_nums == kernel_size)[0][0]
    image_padded = np.zeros((np.shape(image) + 2*pad_val))
    image_padded[pad_val:-pad_val,pad_val:-pad_val] = image
    return image_padded   

def sliding_window(image,window_size,stride,func):
    # grab image size, set container for results
    image_size = np.shape(image)[0]
    results = []
    
    # slide window over input image with given window size / stride and function
    for i in np.arange(0, image_size - window_size + 1, stride):
        for j in np.arange(0, image_size - window_size + 1, stride):
            # now we have a window from our image, and use the desired 'func' to process it
            window = image[i:i+window_size,j:j+window_size]
            
            # process using input func
            processed_window = func(window)
            results.append(processed_window)
    
    # array-afy results
    results = np.array(results)
    
    # return results in numpy array format
    return results

def make_feature_map(image,kernel):
    # pad image appropriately
    kernel_size = kernel.shape[0]
    padded_image = pad_image(image,kernel_size)
    
    # create feature map via convolution --> returns flattened convolution calculations
    conv_stride = 1
    feature_map = sliding_window(padded_image,kernel_size,conv_stride,conv_function)
    
    # re-shape convolution output ---> to square of same size as original input
    new_size = int(np.size(feature_map)**(0.5))
    feature_map = np.reshape(feature_map,(new_size,new_size))
    
    # shove feature map through nonlinearity
    feature_map = activation(feature_map)

    # pool feature map --- i.e., downsample it
    pool_window_size = 6
    pool_stride = 3
    downsampled_feature_map = sliding_window(feature_map,pool_window_size,pool_stride,pool_function)
 
    # return downsampled feature map --> flattened
    return downsampled_feature_map

In [80]:
kernels = np.array([
       [[-1, -1, -1],
        [ 0,  0,  0],
        [ 1,  1,  1]],

       [[-1, -1,  0],
        [-1,  0,  1],
        [ 0,  1,  1]],
    
        [[-1,  0,  1],
        [-1,  0,  1],
        [-1,  0,  1]],

       [[ 0,  1,  1],
        [-1,  0,  1],
        [-1, -1,  0]],

       [[ 1,  0, -1],
        [ 1,  0, -1],
        [ 1,  0, -1]],

       [[ 0, -1, -1],
        [ 1,  0, -1],
        [ 1,  1,  0]],

       [[ 1,  1,  1],
        [ 0,  0,  0],
        [-1, -1, -1]],

       [[ 1,  1,  0],
        [ 1,  0, -1],
        [ 0, -1, -1]]])
                   

In [86]:
kernel = kernels[0]
all_feature_maps = []
for image in image_tensor:
    current_feat_maps = []
    for kernel in kernels:
        # compute feature map for current image using current convolution kernel
        feat_map = make_feature_map(image,kernel)

        # store feature maps of current kernel
        current_feat_maps.append(feat_map)
    
    # append all feature maps from current kernel to running list
    all_feature_maps.append(current_feat_maps)

# convert to array and re-shape properly
all_feature_maps = np.array(all_feature_maps)
all_feature_maps = np.reshape(all_feature_maps,(np.shape(all_feature_maps)[0],np.shape(all_feature_maps)[1]*np.shape(all_feature_maps)[2]),order = 'F')